# Computer Vision Nanodegree

## Project: Image Captioning

In this notebook, some template code has already been provided for you ...

---

### Introduction

xyz

### Step 1: Training Setup

do NOT change the lines of code that are not preceded with a TODO statement.

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

def to_var(x, volatile=False):
    """ converts a Pytorch Tensor to a variable and moves to GPU if CUDA is available """
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

## TODO: Select appropriate values for the Python variables below.
batch_size = 128
vocab_threshold = 4
embed_size = 256
hidden_size = 512
learning_rate = 0.001
num_epochs = 5
save_every = 1

# TODO: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.linear.parameters()) 

# TODO: Define the optimizer.
optimizer = torch.optim.Adam(params=params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.61s)
creating index...


  0%|          | 765/414113 [00:00<00:54, 7648.80it/s]

index created!
Obtaining caption lengths ...


100%|██████████| 414113/414113 [00:41<00:00, 10091.28it/s]


### Step 2: Train your Model!

In [ ]:
# (Optional) TODO: Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))

In [3]:
import torch.utils.data as data
import numpy as np
import os

for epoch in range(num_epochs):
    
    for i_step in range(0, total_step):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        for batch in data_loader:
            images, captions = batch[0], batch[1]
            break 
        
        # Convert batch of images and captions to Pytorch Variable.
        images = to_var(images, volatile=True)
        captions = to_var(captions)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Print training statistics.
        print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f'
            %(epoch+1, num_epochs, i_step, total_step, loss.data[0], np.exp(loss.data[0]))) 
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' %(epoch+1)))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' %(epoch+1)))

Epoch [1/5], Step [0/3236], Loss: 4.9002, Perplexity: 134.3109
Epoch [1/5], Step [1/3236], Loss: 4.8436, Perplexity: 126.9221
Epoch [1/5], Step [2/3236], Loss: 4.6813, Perplexity: 107.9077
Epoch [1/5], Step [3/3236], Loss: 4.8167, Perplexity: 123.5618
Epoch [1/5], Step [4/3236], Loss: 4.8428, Perplexity: 126.8201
Epoch [1/5], Step [5/3236], Loss: 4.8764, Perplexity: 131.1607
Epoch [1/5], Step [6/3236], Loss: 4.7246, Perplexity: 112.6883
Epoch [1/5], Step [7/3236], Loss: 4.6746, Perplexity: 107.1911
Epoch [1/5], Step [8/3236], Loss: 4.5919, Perplexity: 98.6832
Epoch [1/5], Step [9/3236], Loss: 4.6595, Perplexity: 105.5839
Epoch [1/5], Step [10/3236], Loss: 4.7146, Perplexity: 111.5665
Epoch [1/5], Step [11/3236], Loss: 5.6239, Perplexity: 276.9793
Epoch [1/5], Step [12/3236], Loss: 4.6107, Perplexity: 100.5535
Epoch [1/5], Step [13/3236], Loss: 4.5215, Perplexity: 91.9734
Epoch [1/5], Step [14/3236], Loss: 4.5209, Perplexity: 91.9163
Epoch [1/5], Step [15/3236], Loss: 4.5866, Perplexity

KeyboardInterrupt: 